### Import

In [1]:
import numpy as np
import math
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torchtext import data
from tqdm import tnrange, tqdm_notebook
from collections import Counter
from data_loader import DataLoader

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix 
import pickle


%load_ext autoreload
%autoreload 2


### Load Data

In [2]:
data_loader = DataLoader()
train, valid = data_loader.small_train_valid()
train, adversary = data_loader.small_train_valid()

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

loading data...
downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:09<00:00, 8.54MB/s]
.vector_cache/glove.6B.zip: 0.00B [00:00, ?B/s]

splitting data...
building vocabulary...


.vector_cache/glove.6B.zip: 862MB [01:37, 8.87MB/s]                              
100%|█████████▉| 398523/400000 [00:20<00:00, 19776.51it/s]

splitting data...
building vocabulary...


In [3]:
print('%d training examples' %len(train))
print('%d validation examples' %len(valid))


1250 training examples
1250 validation examples


## Naive Bayes - Unigram 

In [4]:
def build_dictionary(data, size):
    """
    Build word count dictionary from training data.
    Return the most common words list () based on input size 
    e.g. [('the', 15969), ('and', 8108), ('is', 5359)...], return type: list
    
    """
    dictionary = Counter()

    for i in range(len(data)):
        for word in data[i].text:
            dictionary[word] += 1
        
    return dictionary.most_common(size)

In [5]:
def extract_features(data, dictionary):
    
    """
    Build word count dictionary from training data.
    Return the most common words based on input size 
    """
    
    docID = 0
    features_matrix = np.zeros((len(data),len(dictionary)))
    label = []

    for i in range(len(data)):
        if data[i].label == ['pos']:
            label.append(1)
        else:
            label.append(0)
        
        for idx, d in enumerate(dictionary):
            wordID = idx
            features_matrix[docID, wordID] += data[i].text.count(d[0])
        docID = docID + 1
        
    return features_matrix, np.array(label)


In [6]:
uniNBdict = build_dictionary(train, 20000)

In [7]:
train_x, train_y = extract_features(train, uniNBdict)
test_x, test_y = extract_features(valid, uniNBdict)

100%|█████████▉| 398523/400000 [00:40<00:00, 19776.51it/s]

In [8]:
uniNB = MultinomialNB()
uniNB.fit(train_x, train_y)

print("unigram - train dataset accuracy", uniNB.score(train_x, train_y))
print("unigram - valid dataset accuracy", uniNB.score(test_x, test_y))


unigram - train dataset accuracy 0.9776
unigram - valid dataset accuracy 0.7936


In [9]:
print(confusion_matrix(test_y, uniNB.predict(test_x)))

[[548  69]
 [189 444]]


## Naive Bayes - Bigrams

In [10]:
def bi_dictionary(data, size):
    
    """
    Build bigram count dictionary from training data.
    Return the most common words list () based on input size 
    e.g. [('of the', 1856), ('. STOP', 880), ('this movie', 753)...], return type: list
    
    """
    
    dictionary = Counter()

    for i in range(len(data)):
        
        bidata = data[i].text.copy()
        bidata.insert(0,"START")
        bidata.append("STOP")
        
        for j in range(len(bidata)-1):  
            bigrams = str(bidata[j]) + " " + str(bidata[j+1])
            dictionary[bigrams] += 1
    
    return dictionary.most_common(size)


In [11]:
def build_dict(text):
    
    """
    Build bigram count dictionary from text.
    e.g. {'the story': 1, 'thinking and': 1, 'this movie': 2 ... }, return type: dictionary
    """
    bi_dict = Counter()
    
    bitext = text.copy()
    bitext.insert(0,"START")
    bitext.append("STOP")
    
    for i in range(len(bitext) -1 ):
        bigrams = str(bitext[i]) + " " + str(bitext[i+1])
        bi_dict[bigrams] += 1
    
    return bi_dict

In [12]:
def bi_extract_features(data, dictionary):

    docID = 0
    features_matrix = np.zeros((len(data),len(dictionary)))
    label = []

    for i in range(len(data)):
        
        if data[i].label == ['pos']:
            label.append(1)
        else:
            label.append(0)
            
        bi_dict = {}
        bi_dict = build_dict(data[i].text)
        
        for idx, d in enumerate(dictionary):
            wordID = idx
            if d[0] in bi_dict:
                features_matrix[docID, wordID] += bi_dict[d[0]]
        docID = docID + 1
        
    return features_matrix, np.array(label)

In [13]:
biNB_dict = bi_dictionary(train, 20000)

In [14]:
train_x_bi, train_y_bi = bi_extract_features(train, biNB_dict)
test_x_bi, test_y_bi = bi_extract_features(valid, biNB_dict)

In [15]:
biNB = MultinomialNB()
biNB.fit(train_x_bi, train_y_bi)
print("bigram - train dataset accuracy", biNB.score(train_x_bi, train_y_bi))
print("bigram - valid dataset accuracy", biNB.score(test_x_bi, test_y_bi))

bigram - train dataset accuracy 0.9912
bigram - valid dataset accuracy 0.8128


In [16]:
print(confusion_matrix(test_y_bi, biNB.predict(test_x_bi)))

[[527  90]
 [144 489]]


## Adversary - swap word (black box)

In [17]:
# Because we perform one swap per word, but do not alter the first or last letters.
# This noise is only applied to words of length > 3.
# noise -> niose

def swap(word):
    rand = random.randint(1,len(word)-3)
    return word[:rand] +  word[rand:rand+2][::-1] + word[rand+2:]

def adversary_swap(adversary_data, propotion):
    """
    Randomly swap characters in words in text, parameter 'propotion' decides how many words are s
    e.g. noise -> niose 
    """
    for i in range(len(adversary_data)):
        rand_sample = random.sample(np.arange(0, len(adversary_data[i].text)).tolist(), int(len(adversary_data[i].text)*propotion))
        for pick in rand_sample:
            if len(adversary_data[i].text[pick]) > 3:
                #print(adversary_data[i].text[pick], "->", swap(adversary_data[i].text[pick]))
                adversary_data[i].text[pick] = swap(adversary_data[i].text[pick])
        #print(" ")
        #print(i, "SWAP END")
        #print(" ")
        
    return adversary_data


In [18]:
adversary_2 = adversary_swap(adversary, 0.8)

### Adversary - Unigram Naive Bayes

In [20]:
adver_x_uni, adver_y_uni = extract_features(adversary_2, uniNBdict)
print("unigram - adversary dataset accuracy", uniNB.score(adver_x_uni, adver_y_uni))
print(confusion_matrix(test_y, uniNB.predict(test_x)))
print(confusion_matrix(adver_y_uni, uniNB.predict(adver_x_uni)))


unigram - adversary dataset accuracy 0.7264
[[548  69]
 [189 444]]
[[541  76]
 [266 367]]


### Adversary - Bigram Naive Bayes

In [21]:
adver_x_bi, adver_y_bi = bi_extract_features(adversary_2, biNB_dict)

In [22]:
print("bigram - adversary dataset accuracy", biNB.score(adver_x_bi, adver_y_bi))

bigram - adversary dataset accuracy 0.7448


In [23]:
print(confusion_matrix(test_y_bi, biNB.predict(test_x_bi)))
print(confusion_matrix(adver_y_bi, biNB.predict(adver_x_bi)))

[[527  90]
 [144 489]]
[[454 163]
 [156 477]]


In [30]:
# save the model to disk
filename = 'uniNB_modle.sav'
pickle.dump(uniNB, open(filename, 'wb'))

filename = 'biNB_model.sav'
pickle.dump(biNB, open(filename, 'wb'))


In [ ]:
len(data_loader.TEXT.vocab.itos)
pretrained_embeddings = data_loader.TEXT.vocab.vectors
len(pretrained_embeddings)